# Order-free matching performance

In [1]:
# Generic imports

import glob
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import ast
import json


# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

from tqdm import tqdm

## Read order-free candidates

In [2]:
picos = 'I' # ['P', 'I', 'O', 'S']
match_level = 'doc' # ['doc', 'sent', 'win_5', 'para']

In [3]:
order_free_dir = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_free/{match_level}/{picos}'
order_free_files = os.listdir(order_free_dir)
print('Files: ', order_free_files)

Files:  ['.nfs0000000011200a4200000003', 'train_ebm_intervention.json', 'train_ebm_intervention_syn.json']


In [4]:
order_free_files.remove('.nfs0000000011200a4200000003')

In [5]:
orf_loaded_files = dict()

for i in tqdm(order_free_files):
    
    filpath = f'{order_free_dir}/{i}'
    print('Loading file...', i)
    with open( filpath, 'r' ) as rf:
        orf_i = json.load(rf)
        orf_loaded_files[i] = orf_i

  0%|          | 0/2 [00:00<?, ?it/s]

Loading file... train_ebm_intervention.json


 50%|█████     | 1/2 [00:13<00:13, 13.77s/it]

Loading file... train_ebm_intervention_syn.json


100%|██████████| 2/2 [00:19<00:00,  9.52s/it]


## difference between intervention and intervention syn

In [6]:
def get_orf(v, par):
    
    orfs = dict()
    
    for k_i, v_i in v.items():
        # print( k_i ) # example: name_15_3, name_9_9, name_12_8
        
        if 'Inters. (full)' in v_i and len(v_i['Inters. (full)']) > 0:
            
            full_inters = v_i['Inters. (full)']
            # full_inters.keys() = PMIDs
            # full_inters.values() = offsets, tokens

            for pmid, matches in full_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        orfs[pmid] = [ ]
                        orfs[pmid].extend( matches['word offs.'] )
                    else:
                        orfs[pmid].extend( matches['word offs.'] )


        if 'Inters. (partial)' in v_i and len(v_i['Inters. (partial)']) > 0 and (par=='both' or par==True):
            
            par_inters = v_i['Inters. (partial)']
            
            for pmid, matches in par_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        orfs[pmid] = [ ]
                        orfs[pmid].extend( matches['word offs.'] )
                    else:
                        orfs[pmid].extend( matches['word offs.'] )
    
    return orfs

In [7]:
orf_int_syn = get_orf(orf_loaded_files['train_ebm_intervention_syn.json'], par = True)
orf_int = get_orf(orf_loaded_files['train_ebm_intervention.json'], par = True)

In [8]:
total_offsets_syn = []
for k,v in orf_int_syn.items():
    total_offsets_syn.extend( v )

In [9]:
len(total_offsets_syn)

107195

In [10]:
total_offsets = []
for k,v in orf_int.items():
    total_offsets.extend( v )

In [11]:
len(total_offsets)

28661

In [12]:
# just full
print(len(total_offsets_syn))
print(len(total_offsets))

107195
28661


## Merge dictionaries

In [13]:
def FullMergeDict(D1, D2):
    
    D_merged = dict()
    
    for k,v in D1.items():
        
        if k not in D_merged:
            D_merged[k] = []
            D_merged[k].extend( v )
        else:
            D_merged[k].extend( v )

    for k,v in D2.items():
        
        if k not in D_merged:
            D_merged[k] = []
            D_merged[k].extend( v )
        else:
            D_merged[k].extend( v )
            
    return D_merged

In [14]:
orfs_merged = FullMergeDict( orf_int, orf_int_syn )

In [15]:
total_offsets_merged = []
for k,v in orfs_merged.items():
    total_offsets_merged.extend( v )
    
print('Total offsets after the int. and int. syn offsets were merged: ', len(total_offsets_merged) )

Total offsets after the int. and int. syn offsets were merged:  135856


## Load order-bound matches

In [16]:
def order_free_matches(x, orf_offsets):
    
    labs_modified = []
    
    for i, (identifier, offs, labs) in enumerate( zip(x.pmid, x.offsets, x.labels) ):
             
        lab_val = [v for k, v in ast.literal_eval(labs).items()] 
        off_val = ast.literal_eval(offs) 
        
        if str(identifier) in orf_offsets: 
            orf_matches =  orf_offsets[ str(identifier) ]
            match_indices = [ off_val.index(m) for m in orf_matches ]
            for i, l in enumerate(lab_val):
                if i in match_indices:
                    lab_val[i] = 1
                    
        labs_modified.append( lab_val )
        
        
    return labs_modified

In [17]:
ob_int = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/direct/{picos}/lf_ds_intervention_syn.tsv'
ob_int_syn = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/direct/{picos}/lf_ds_intervetion.tsv'

In [18]:
ob_int_df = pd.read_csv(ob_int, sep='\t', header=0)
ob_int_syn_df = pd.read_csv(ob_int_syn, sep='\t', header=0)
ob_merged_df = pd.concat([ob_int_df,ob_int_syn_df])

In [19]:
def process_gt(l):
    
    labels = l
    
    if isinstance(labels, str):
        labels = ast.literal_eval(labels)
        
    # convert non-1 fine labels labels to 1's
    labels = ['1' if (n != '1' and n != '0') else str(n) for i, n in enumerate(labels) ]
    
    return labels

ob_int_df['i'] = ob_int_df.i.apply(process_gt)
ob_int_syn_df['i'] = ob_int_syn_df.i.apply(process_gt)

ob_int_df['i_f'] = ob_int_df.i_f.apply(process_gt)
ob_int_syn_df['i_f'] = ob_int_syn_df.i_f.apply(process_gt)

In [20]:
# Fetch ground truth from the direct matching

coarse_int_gt = dict(zip(ob_int_df['pmid'], ob_int_df['i']))
fine_int_gt = dict(zip(ob_int_df['pmid'], ob_int_df['i_f']))

In [21]:
# Preprocess order-bound labels

def process_ob_labs(l):
    
    labels = l
    
    if isinstance( labels, str ):
        labels = ast.literal_eval(labels)

    labels = [ v for k, v in labels.items() ]
    labels = ['0' if n == -1 else str(n) for i, n in enumerate(labels) ]

    return labels

ob_int_df['labels'] = ob_int_df.labels.apply(process_ob_labs) # order bound matching labels for int source
ob_int_syn_df['labels'] = ob_int_syn_df.labels.apply(process_ob_labs) # order bound matching labels for int_syn source

In [22]:
# Fetch order-bound predictions for merged dataframes

ob_preds_merged = dict()

ob_int_dict = dict(zip(ob_int_df['pmid'], ob_int_df['labels']))
ob_int_syn_dict = dict(zip(ob_int_syn_df['pmid'], ob_int_syn_df['labels']))

for k,v in ob_int_dict.items():

    if k not in ob_preds_merged:
        ob_preds_merged[k] = []
        ob_preds_merged[k] = v

    else:
        old_pred = ob_preds_merged[k]
        new_pred = v
        
        # merge old and new predictions
        merged_predictions = [ max( o,n ) for o,n in zip( old_pred, new_pred ) ]
        assert len( old_pred ) == len( new_pred ) == len( merged_predictions )
        ob_preds_merged[k] = merged_predictions

In [23]:
len( list(ob_preds_merged.values()) )

4802

In [24]:
for k,v in ob_int_syn_dict.items():

    if k not in ob_preds_merged:
        ob_preds_merged[k] = []
        ob_preds_merged[k] = v

    else:
        old_pred = ob_preds_merged[k]
        new_pred = v

        # merge old and new predictions
        #print( 'merging the new predictions...' )
        merged_predictions = [ max( o,n ) for o,n in zip( old_pred, new_pred ) ]
        assert len( old_pred ) == len( new_pred ) == len( merged_predictions )
        ob_preds_merged[k] = merged_predictions

In [25]:
total_total = []
for k,v in ob_preds_merged.items():
    total_total.extend( v )
    
print( len(total_total) )

1303169


## Merge ob preds with orf preds

In [26]:
def merge_preds_ordered( ob_preds, orf_offsets ):
    
    merged_predictions = dict()
    
    for k, v in ob_preds.items():
        
        k = str(k)

        if k in orf_offsets:
            matching_offsets = orf_offsets[k]
            old_preds = list( map( int, v ))
            new_preds = list( map( int, v ))
                             
            # add new offsets to the new_preds
            for indice in matching_offsets:
                new_preds[indice] = 1

            merged_predictions[ k ] = new_preds
        else:
            merged_predictions[ k ] = list( map( int, v ) )
    
    return merged_predictions
                             
                             
ob_of_int = merge_preds_ordered( ob_preds_merged, orf_int )
ob_of_int_syn = merge_preds_ordered( ob_preds_merged, orf_int_syn )
ob_of_int_merged = merge_preds_ordered( ob_preds_merged, orfs_merged )

In [27]:
total_total = []
for k,v in ob_of_int_syn.items():
    total_total.extend( v )
    
print( len(total_total) )

1303169


## evaluation metrics

In [28]:
def flatten(d):
    l = [ v for k,v in d.items() ]
    l = [item for sublist in l for item in sublist]
    l = list(map(int, l))
    return l

In [29]:
# ground truth : coarse_int_gt, fine_int_gt
# ob/direct matching preds :  ob_preds_merged

order_bound_preds = flatten( ob_preds_merged )
picos_coarse = flatten( coarse_int_gt )
picos_fine = flatten( fine_int_gt )

##### direct matching

In [30]:
# Classification report
cr_order_bound_coarse = classification_report( picos_coarse, order_bound_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and order-bound matches')
print( cr_order_bound_coarse )

cr_order_bound_fine = classification_report( picos_fine, order_bound_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and order-bound matches')
print( cr_order_bound_fine )

Confusion matrix for coarse-grained ground truth and order-bound matches
              precision    recall  f1-score   support

           0     0.9295    0.7394    0.8236   1177209
           1     0.1635    0.4761    0.2434    125960

    accuracy                         0.7139   1303169
   macro avg     0.5465    0.6077    0.5335   1303169
weighted avg     0.8555    0.7139    0.7675   1303169



Confusion matrix for fine-grained ground truth and order-bound matches
              precision    recall  f1-score   support

           0     0.9565    0.7385    0.8335   1212904
           1     0.1352    0.5492    0.2169     90265

    accuracy                         0.7254   1303169
   macro avg     0.5459    0.6438    0.5252   1303169
weighted avg     0.8997    0.7254    0.7908   1303169



#### ORF matching - int

In [31]:
# order free matching preds - int : ob_of_int
# order free matching preds - int_syn : ob_of_int_syn
# order free matching preds - merged : ob_of_int_merged

In [32]:
ob_of_int_preds = flatten( ob_of_int )
ob_of_int_syn_preds = flatten( ob_of_int_syn )
ob_of_int_merged_preds = flatten( ob_of_int_merged )

In [33]:
# Classification report
cr_orf_int_coarse = classification_report( picos_coarse, ob_of_int_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and (int.) document level order-free matches')
print( cr_orf_int_coarse )

cr_orf_int_fine = classification_report( picos_fine, ob_of_int_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and (int.) document level order-free matches')
print( cr_orf_int_fine )

Confusion matrix for coarse-grained ground truth and (int.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9295    0.7369    0.8221   1177209
           1     0.1627    0.4780    0.2428    125960

    accuracy                         0.7119   1303169
   macro avg     0.5461    0.6074    0.5324   1303169
weighted avg     0.8554    0.7119    0.7661   1303169



Confusion matrix for fine-grained ground truth and (int.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9566    0.7360    0.8319   1212904
           1     0.1345    0.5512    0.2162     90265

    accuracy                         0.7232   1303169
   macro avg     0.5455    0.6436    0.5241   1303169
weighted avg     0.8996    0.7232    0.7893   1303169



In [34]:
# Classification report
cr_orf_intsyn_coarse = classification_report( picos_coarse, ob_of_int_syn_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and (int syn.) document level order-free matches')
print( cr_orf_intsyn_coarse )

cr_orf_intsyn_fine = classification_report( picos_fine, ob_of_int_syn_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and (int syn.) document level order-free matches')
print( cr_orf_intsyn_fine )

Confusion matrix for coarse-grained ground truth and (int syn.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9296    0.7333    0.8199   1177209
           1     0.1617    0.4807    0.2420    125960

    accuracy                         0.7089   1303169
   macro avg     0.5456    0.6070    0.5309   1303169
weighted avg     0.8554    0.7089    0.7640   1303169



Confusion matrix for fine-grained ground truth and (int syn.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9566    0.7325    0.8297   1212904
           1     0.1335    0.5539    0.2152     90265

    accuracy                         0.7201   1303169
   macro avg     0.5451    0.6432    0.5224   1303169
weighted avg     0.8996    0.7201    0.7871   1303169



In [35]:
# Classification report
cr_orf_intmerged_coarse = classification_report( picos_coarse, ob_of_int_merged_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and (merged.) document level order-free matches')
print( cr_orf_intmerged_coarse )

cr_orf_intmerged_fine = classification_report( picos_fine, ob_of_int_merged_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and (merged.) document level order-free matches')
print( cr_orf_intmerged_fine )

Confusion matrix for coarse-grained ground truth and (merged.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9296    0.7314    0.8187   1177209
           1     0.1611    0.4821    0.2415    125960

    accuracy                         0.7073   1303169
   macro avg     0.5453    0.6068    0.5301   1303169
weighted avg     0.8553    0.7073    0.7629   1303169



Confusion matrix for fine-grained ground truth and (merged.) document level order-free matches
              precision    recall  f1-score   support

           0     0.9567    0.7306    0.8285   1212904
           1     0.1330    0.5553    0.2146     90265

    accuracy                         0.7184   1303169
   macro avg     0.5448    0.6429    0.5215   1303169
weighted avg     0.8996    0.7184    0.7859   1303169

